In [3]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')


In [4]:
login = pd.read_csv('key.csv')
consumer_key = login['key'][0]
consumer_secret = login['secret'][0]
access_token = login['token'][0]
access_token_secret = login['token_secret'][0]

In [5]:
#auth obj
authenticate = tweepy.OAuthHandler(consumer_key, consumer_secret)

#set access token
authenticate.set_access_token(access_token, access_token_secret)

#create api obj
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [12]:
#Extract 100 tweets from twitter user
def getTweetFromUser(query,n,lang):
    return api.user_timeline(screen_name=query, count=n, lang=lang, tweet_mode="extended")

def getTweet(query,n):
    return api.search(q=query, count=n, tweet_mode="extended")

posts = getTweet("#ม็อบ18พฤศจิกา",100)


for tweet in posts[0:5]:
    print(tweet.full_text+'\n')

RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…

RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…

RT @guykrissssss: เฌอเอมปังมาก!!! #ม็อบ18พฤศจิกา https://t.co/xXVdAqhXjc

RT @oilSGTH: “ใครที่ไม่อยากโดนสี จริงๆกูก็ไม่อยากโดนอ่ามึงเพราะว่าเสื้อมันแพงนิดนึง” 55555555555555555555555555
#ม็อบ18พฤศจิกา https://t.co…

RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…



In [13]:
#create dataframe
df = pd.DataFrame([tweet.full_text for tweet in posts], columns= ['Tweets'])

df.head()

,Tweets
0,RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ...
1,RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ...
2,RT @guykrissssss: เฌอเอมปังมาก!!! #ม็อบ18พฤศจิ...
3,RT @oilSGTH: “ใครที่ไม่อยากโดนสี จริงๆกูก็ไม่อ...
4,RT @zkdk_92: ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ...


In [14]:
#clean the text

#create function to clean the tweets
def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9_]+:', '', text) # remove #mentions
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # remove #mentions
    text = re.sub(r'#','',text) # remove hashtag
    text = re.sub(r'RT[\s]+','', text) # remove RT
    text = re.sub(r'https?:\/\/\S+','', text) # remove hyper link
    return text

#clean the text
df['Tweets'] = df['Tweets'].apply(cleanText)

df

,Tweets
0,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...
1,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...
2,เฌอเอมปังมาก!!! ม็อบ18พฤศจิกา
3,“ใครที่ไม่อยากโดนสี จริงๆกูก็ไม่อยากโดนอ่ามึงเ...
4,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...
...,...
95,"ไม่เข้าใจว่าป้ายนี้หมายถึงอะไร?\n\n""อู๊ดกลัวเป..."
96,คนจริงๆเต็มราชประสงค์ไม่ใช่ภาพตัดต่อไม่ต้องใช้...
97,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...
98,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...


In [15]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

#create net col
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

In [16]:
df

,Tweets,Subjectivity,Polarity
0,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...,0.0,0.0
1,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...,0.0,0.0
2,เฌอเอมปังมาก!!! ม็อบ18พฤศจิกา,0.0,0.0
3,“ใครที่ไม่อยากโดนสี จริงๆกูก็ไม่อยากโดนอ่ามึงเ...,0.0,0.0
4,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...,0.0,0.0
...,...,...,...
95,"ไม่เข้าใจว่าป้ายนี้หมายถึงอะไร?\n\n""อู๊ดกลัวเป...",0.0,0.0
96,คนจริงๆเต็มราชประสงค์ไม่ใช่ภาพตัดต่อไม่ต้องใช้...,0.0,0.0
97,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...,0.0,0.0
98,ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิด...,0.0,0.0


In [17]:
df.to_csv(r'sentiment.csv', index = False)

In [21]:
df.to_numpy()

array([['ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…',
        0.0, 0.0],
       ['ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…',
        0.0, 0.0],
       ['เฌอเอมปังมาก!!! ม็อบ18พฤศจิกา ', 0.0, 0.0],
       ['“ใครที่ไม่อยากโดนสี จริงๆกูก็ไม่อยากโดนอ่ามึงเพราะว่าเสื้อมันแพงนิดนึง” 55555555555555555555555555\nม็อบ18พฤศจิกา ',
        0.0, 0.0],
       ['ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…',
        0.0, 0.0],
       ['ฝากรีหน่อยนะคะจำน้องๆกันได้มั้ยคะ ขออนุญาตเปิดรับบริจาคอีกครั้ง ตอนนี้น้องๆ ต้องการอาหารสำหรับของเดือนหน้าอยู่นะคะ ตอนนี้มีจำน…',
        0.0, 0.0],
       ['ดูคลิปแล้วทั้งหดหู่ทั้งเจ็บใจ บวกกับเมื่อกี้คุยกับแท็กซี่ เค้าบอกข้าวทำปีละครั้ง ทำแต่นาปี เพราะนาปรังต้นแค่เข่า ทุกคนเลย…',
        0.0, 0.0],
       ['おはようございま